In [62]:
import numpy as np
import pandas as pd
from scipy.stats import erlang, expon
import matplotlib.pyplot as plt
import math
import operator

import obnetwork2

In [3]:
%matplotlib inline

In [15]:
output_stats_summary_df = pd.read_csv('./output/output_stats_summary.csv').sort_values(by=['scenario', 'rep'])
output_stats_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3750 entries, 0 to 1442
Data columns (total 78 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   scenario                3750 non-null   int64  
 1   rep                     3750 non-null   int64  
 2   timestamp               3750 non-null   object 
 3   num_days                3750 non-null   float64
 4   num_visits_obs          3750 non-null   float64
 5   num_visits_ldr          3750 non-null   float64
 6   num_visits_pp           3750 non-null   float64
 7   num_visits_csect        3750 non-null   float64
 8   planned_los_mean_obs    3750 non-null   float64
 9   actual_los_mean_obs     3750 non-null   float64
 10  planned_los_sd_obs      3750 non-null   float64
 11  actual_los_sd_obs       3750 non-null   float64
 12  planned_los_cv2_obs     3750 non-null   float64
 13  actual_los_cv2_obs      3750 non-null   float64
 14  planned_los_skew_obs    3750 non-null   

In [18]:
output_stats_summary_agg_df = output_stats_summary_df.groupby(['scenario']).agg(
    num_visits_obs_mean=pd.NamedAgg(column='num_visits_obs', aggfunc='mean'),
    num_visits_ldr_mean=pd.NamedAgg(column='num_visits_ldr', aggfunc='mean'),
    num_visits_csect_mean=pd.NamedAgg(column='num_visits_csect', aggfunc='mean'),
    num_visits_pp_mean=pd.NamedAgg(column='num_visits_pp', aggfunc='mean'),

    planned_los_mean_mean_obs=pd.NamedAgg(column='planned_los_mean_obs', aggfunc='mean'),
    planned_los_mean_mean_ldr=pd.NamedAgg(column='planned_los_mean_ldr', aggfunc='mean'),
    planned_los_mean_mean_csect=pd.NamedAgg(column='planned_los_mean_csect', aggfunc='mean'),
    planned_los_mean_mean_pp=pd.NamedAgg(column='planned_los_mean_pp', aggfunc='mean'),
    
    actual_los_mean_mean_obs=pd.NamedAgg(column='actual_los_mean_obs', aggfunc='mean'),
    actual_los_mean_mean_ldr=pd.NamedAgg(column='actual_los_mean_ldr', aggfunc='mean'),
    actual_los_mean_mean_csect=pd.NamedAgg(column='actual_los_mean_csect', aggfunc='mean'),
    actual_los_mean_mean_pp=pd.NamedAgg(column='actual_los_mean_pp', aggfunc='mean'),
    
    planned_los_mean_cv2_obs=pd.NamedAgg(column='planned_los_cv2_obs', aggfunc='mean'),
    planned_los_mean_cv2_ldr=pd.NamedAgg(column='planned_los_cv2_ldr', aggfunc='mean'),
    planned_los_mean_cv2_csect=pd.NamedAgg(column='planned_los_cv2_csect', aggfunc='mean'),
    planned_los_mean_cv2_pp=pd.NamedAgg(column='planned_los_cv2_pp', aggfunc='mean'),
    
    actual_los_mean_cv2_obs=pd.NamedAgg(column='actual_los_cv2_obs', aggfunc='mean'),
    actual_los_mean_cv2_ldr=pd.NamedAgg(column='actual_los_cv2_ldr', aggfunc='mean'),
    actual_los_mean_cv2_csect=pd.NamedAgg(column='actual_los_cv2_csect', aggfunc='mean'),
    actual_los_mean_cv2_pp=pd.NamedAgg(column='actual_los_cv2_pp', aggfunc='mean'),
    
    occ_mean_mean_obs=pd.NamedAgg(column='occ_mean_obs', aggfunc='mean'),
    occ_mean_mean_ldr=pd.NamedAgg(column='occ_mean_ldr', aggfunc='mean'),
    occ_mean_mean_csect=pd.NamedAgg(column='occ_mean_csect', aggfunc='mean'),
    occ_mean_mean_pp=pd.NamedAgg(column='occ_mean_pp', aggfunc='mean'),

    occ_mean_p95_obs=pd.NamedAgg(column='occ_p95_obs', aggfunc='mean'),
    occ_mean_p95_ldr=pd.NamedAgg(column='occ_p95_ldr', aggfunc='mean'),
    occ_mean_p95_csect=pd.NamedAgg(column='occ_p95_csect', aggfunc='mean'),
    occ_mean_p95_pp=pd.NamedAgg(column='occ_p95_pp', aggfunc='mean'),
    
    mean_pct_waitq_ldr=pd.NamedAgg(column='pct_waitq_ldr', aggfunc='mean'),
    mean_waitq_ldr_mean=pd.NamedAgg(column='waitq_ldr_mean', aggfunc='mean'),
    mean_waitq_ldr_p95=pd.NamedAgg(column='waitq_ldr_p95', aggfunc='mean'), 

    mean_pct_blocked_by_pp=pd.NamedAgg(column='pct_waitq_ldr', aggfunc='mean'),
    mean_blocked_by_pp_mean=pd.NamedAgg(column='blocked_by_pp_mean', aggfunc='mean'),
    mean_blocked_by_pp_p95=pd.NamedAgg(column='blocked_by_pp_p95', aggfunc='mean'),
)

In [23]:
output_stats_summary_agg_df.to_csv('./output/output_stats_summary_agg.csv', index=True)

Now compute queueing approximations

In [27]:
scenario_summary_stats_df = pd.read_csv('./output/output_stats_summary_agg.csv')
input_params_df = pd.read_csv('./inputs/exp10_tandem05_metainputs.csv')
#train_df.set_index('scenario', drop=False, inplace=True)

In [26]:
scenario_summary_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   scenario                     150 non-null    int64  
 1   num_visits_obs_mean          150 non-null    float64
 2   num_visits_ldr_mean          150 non-null    float64
 3   num_visits_csect_mean        150 non-null    float64
 4   num_visits_pp_mean           150 non-null    float64
 5   planned_los_mean_mean_obs    150 non-null    float64
 6   planned_los_mean_mean_ldr    150 non-null    float64
 7   planned_los_mean_mean_csect  150 non-null    float64
 8   planned_los_mean_mean_pp     150 non-null    float64
 9   actual_los_mean_mean_obs     150 non-null    float64
 10  actual_los_mean_mean_ldr     150 non-null    float64
 11  actual_los_mean_mean_csect   150 non-null    float64
 12  actual_los_mean_mean_pp      150 non-null    float64
 13  planned_los_mean_cv2

In [28]:
input_params_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   scenario                 150 non-null    int64  
 1   arrival_rate             150 non-null    float64
 2   mean_los_obs             150 non-null    float64
 3   num_erlang_stages_obs    150 non-null    int64  
 4   mean_los_ldr             150 non-null    int64  
 5   num_erlang_stages_ldr    150 non-null    int64  
 6   mean_los_csect           150 non-null    int64  
 7   num_erlang_stages_csect  150 non-null    int64  
 8   mean_los_pp_noc          150 non-null    int64  
 9   mean_los_pp_c            150 non-null    int64  
 10  num_erlang_stages_pp     150 non-null    int64  
 11  c_sect_prob              150 non-null    float64
 12  tot_c_rate               150 non-null    float64
 13  acc_tar_obs              150 non-null    float64
 14  acc_tar_ldr              1

In [35]:
input_params_df.iloc[:, 5:15].head()

,num_erlang_stages_ldr,mean_los_csect,num_erlang_stages_csect,mean_los_pp_noc,mean_los_pp_c,num_erlang_stages_pp,c_sect_prob,tot_c_rate,acc_tar_obs,acc_tar_ldr
0,2,1,4,48,72,8,0.2,0.2,0.99995,0.95
1,2,1,4,48,72,8,0.2,0.2,0.99995,0.95
2,2,1,4,48,72,8,0.2,0.2,0.99995,0.95
3,2,1,4,48,72,8,0.2,0.2,0.99995,0.85
4,2,1,4,48,72,8,0.2,0.2,0.99995,0.85


In [29]:
obsim_mm_means_df = input_params_df.merge(scenario_summary_stats_df, on='scenario')

In [30]:
obsim_mm_means_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   scenario                     150 non-null    int64  
 1   arrival_rate                 150 non-null    float64
 2   mean_los_obs                 150 non-null    float64
 3   num_erlang_stages_obs        150 non-null    int64  
 4   mean_los_ldr                 150 non-null    int64  
 5   num_erlang_stages_ldr        150 non-null    int64  
 6   mean_los_csect               150 non-null    int64  
 7   num_erlang_stages_csect      150 non-null    int64  
 8   mean_los_pp_noc              150 non-null    int64  
 9   mean_los_pp_c                150 non-null    int64  
 10  num_erlang_stages_pp         150 non-null    int64  
 11  c_sect_prob                  150 non-null    float64
 12  tot_c_rate                   150 non-null    float64
 13  acc_tar_obs         

In [71]:
def hyper_erlang_moment(rates, stages, probs, moment):
    terms = [probs[i - 1] * math.factorial(stages[i - 1] + moment - 1) * ( 1 / math.factorial(stages[i - 1] - 1)) * (stages[i - 1] * rates[i - 1]) ** (-moment) 
            for i in range(1, len(rates) + 1)]
    
    return sum(terms)

In [77]:
results = []
scenarios = range(1,151)

for row in obsim_mm_means_df.iterrows():

    scenario = row[1]['scenario']
    arr_rate = row[1]['arrival_rate']
    c_sect_prob = row[1]['c_sect_prob']
    ldr_mean_svctime = row[1]['mean_los_ldr']
    ldr_cv2_svctime = 1 / row[1]['num_erlang_stages_ldr']
    ldr_cap = row[1]['cap_ldr']
    pp_mean_svctime = c_sect_prob * row[1]['mean_los_pp_c'] + (1 - c_sect_prob) * row[1]['mean_los_pp_noc']
    
    rates = [1 / row[1]['mean_los_pp_c'], 1 / row[1]['mean_los_pp_noc']]
    probs = [c_sect_prob, 1 - c_sect_prob]
    stages = [int(row[1]['num_erlang_stages_pp']), int(row[1]['num_erlang_stages_pp'])]
    moments = [hyper_erlang_moment(rates, stages, probs, moment) for moment in [1, 2]]
    variance = moments[1] - moments[0] ** 2
    cv2 = variance / moments[0] ** 2
    
    pp_cv2_svctime = cv2
    
    pp_cap = row[1]['cap_pp']
    sim_mean_waitq_ldr_mean = row[1]['mean_waitq_ldr_mean']
    sim_mean_pct_waitq_ldr = row[1]['mean_pct_waitq_ldr']
    sim_actual_los_mean_mean_ldr = row[1]['actual_los_mean_mean_ldr']
    sim_mean_pct_blocked_by_pp = row[1]['mean_pct_blocked_by_pp']
    sim_mean_blocked_by_pp_mean = row[1]['mean_blocked_by_pp_mean']

    ldr_pct_blockedby_pp = obnetwork2.ldr_prob_blockedby_pp_hat(arr_rate, pp_mean_svctime, pp_cap, pp_cv2_svctime)
    ldr_meantime_blockedby_pp = obnetwork2.ldr_condmeantime_blockedby_pp_hat(arr_rate, pp_mean_svctime, pp_cap, pp_cv2_svctime)
    (obs_meantime_blockedbyldr, ldr_effmean_svctime, obs_prob_blockedby_ldr, obs_condmeantime_blockedbyldr) = \
        obnetwork2.obs_blockedby_ldr_hats(arr_rate, c_sect_prob, ldr_mean_svctime, ldr_cv2_svctime, ldr_cap,
                               pp_mean_svctime, pp_cv2_svctime, pp_cap)

    scen_results = {'scenario': scenario,
                    'arr_rate': arr_rate,
                    'prob_blockedby_ldr_approx': obs_prob_blockedby_ldr,
                    'prob_blockedby_ldr_sim': sim_mean_pct_waitq_ldr,

                    'condmeantime_blockedbyldr_approx': obs_condmeantime_blockedbyldr,
                    'condmeantime_blockedbyldr_sim': sim_mean_waitq_ldr_mean,
                    'ldr_effmean_svctime_approx': ldr_effmean_svctime,
                    'ldr_effmean_svctime_sim': sim_actual_los_mean_mean_ldr,
                    'prob_blockedby_pp_approx': ldr_pct_blockedby_pp,
                    'prob_blockedby_pp_sim': sim_mean_pct_blocked_by_pp,
                    'condmeantime_blockedbypp_approx': ldr_meantime_blockedby_pp,
                    'condmeantime_blockedbypp_sim': sim_mean_blocked_by_pp_mean}

    results.append(scen_results)

    # print("scenario {}\n".format(scenario))
    # print(results)


results_df = pd.DataFrame(results)
print(results_df)

#results_df.to_csv("obnetwork_approx_vs_sim.csv")
results_df.to_csv("./output/obnetwork_approx_vs_sim_testing.csv")

     scenario  arr_rate  prob_blockedby_ldr_approx  prob_blockedby_ldr_sim  \
0         1.0  0.114155                   0.055086                0.051794   
1         2.0  0.114155                   0.063771                0.064413   
2         3.0  0.114155                   0.080531                0.086950   
3         4.0  0.114155                   0.165820                0.163261   
4         5.0  0.114155                   0.182374                0.181308   
..        ...       ...                        ...                     ...   
145     146.0  0.799087                   0.000009                0.000006   
146     147.0  0.799087                   0.000009                0.000003   
147     148.0  1.027397                   0.000006                0.000000   
148     149.0  1.027397                   0.000006                0.000004   
149     150.0  1.027397                   0.000006                0.000004   

     condmeantime_blockedbyldr_approx  condmeantime_blockedbyld

In [67]:
means = [48, 72]
rates = [1 / mean for mean in means]
probs = [0.8, 0.2]
stages = [8, 8]
moment = 1

In [68]:
hyper_erlang_moment(rates, stages, probs, moment)

[38.400000000000006, 14.4]


52.800000000000004

In [73]:
moments = [hyper_erlang_moment(rates, stages, probs, moment) for moment in [1, 2]]

In [74]:
variance = moments[1] - moments[0] ** 2
variance

452.1600000000003

In [75]:
cv2 = variance / moments[0] ** 2
cv2

0.16219008264462817

In [63]:
def sumproduct(*lists):  return sum(map(operator.mul, *lists))

In [69]:
sumproduct(means, probs)

52.800000000000004

In [47]:
math.factorial(4)

24

In [48]:
-2

-2

In [49]:
x = 3
-x

-3